In [ ]:

import joblib
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

# 1. Charger le modèle et les données

try:
    model = joblib.load("xgboost_power_predictor_optimized.pkl")
    print("✅ Modèle chargé avec succès.")
except FileNotFoundError:
    print("❌ Erreur : Fichier modèle 'xgboost_power_predictor_optimized.pkl' introuvable.")
    exit()

try:
    df = pd.read_csv("sim_power_data_enhanced.csv")
    df = df.replace([np.inf, -np.inf], np.nan).dropna()
    print("✅ Dataset de simulation chargé.")
except FileNotFoundError:
    print("❌ Erreur : Fichier de données 'sim_power_data_enhanced.csv' introuvable.")
    exit()

# 2. Recréer le jeu de test pour obtenir des données réalistes

features = [
    'orb_phase', 'eclipse_flag', 'sun_incidence_cos', 'solar_irradiance_Wm2',
    'panel_temp_C', 'P_panel_pred', 'mppt_eff', 'payload_flag',
    'P_payload_pred', 'P_base_pred', 'SoC',
    'P_panel_pred_rolling_mean',
    'P_panel_pred_rolling_std',
    'panel_temp_C_rolling_mean',
    'irradiance_x_mppt'
]
target = 'P_future_720s'

X = df[features]
y = df[target]
_, X_test, _, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


# 3. Choisir un scénario de test aléatoire


random_index = np.random.randint(0, len(X_test))
current_simulated_state = X_test.iloc[[random_index]]
true_future_power = y_test.iloc[random_index]


# 4. Faire une prédiction et afficher le résultat


predicted_power_array = model.predict(current_simulated_state)
predicted_power = predicted_power_array[0] 
print("\n=======================================================")
print(f"🔄 Test avec le scénario N°{random_index} du jeu de test...")
print("-------------------------------------------------------")
print("📊 État 'actuel' simulé (données d'entrée) :")
print(current_simulated_state.T.to_string(header=False))
print("-------------------------------------------------------")
print(f"⚡ Votre puissance disponible sera de {predicted_power:.2f} W dans 12 minutes.")
print(f"🎯 Pour comparaison, la valeur réelle était : {true_future_power:.2f} W")
print(f"   (Erreur sur cet exemple : {abs(predicted_power - true_future_power):.2f} W)")


✅ Modèle chargé avec succès.
✅ Dataset de simulation chargé.

🔄 Test avec le scénario N°1084 du jeu de test...
-------------------------------------------------------
📊 État 'actuel' simulé (données d'entrée) :
orb_phase                    4.652699
eclipse_flag                 0.000000
sun_incidence_cos            0.529439
solar_irradiance_Wm2       720.566145
panel_temp_C                34.400094
P_panel_pred                18.962882
mppt_eff                     0.895424
payload_flag                 1.000000
P_payload_pred               2.491969
P_base_pred                  5.721468
SoC                          1.000000
P_panel_pred_rolling_mean   19.087910
P_panel_pred_rolling_std    10.980242
panel_temp_C_rolling_mean   36.363195
irradiance_x_mppt          645.212076
-------------------------------------------------------
⚡ Votre puissance disponible sera de 13.42 W dans 12 minutes.
🎯 Pour comparaison, la valeur réelle était : 12.45 W
   (Erreur sur cet exemple : 0.98 W)

